# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [2]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [3]:
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# glimpse at the general population dataset
azdias.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [5]:
# a quick look at the mail-order company customers dataset
customers.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
1,9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
2,143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4
3,143873,1,1.0,8.0,NaN,NaN,NaN,NaN,8.0,0.0,...,2.0,NaN,9.0,7.0,1,COSMETIC,MULTI_BUYER,0,1,4
4,143874,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,7.0,...,4.0,2.0,9.0,3.0,1,FOOD,MULTI_BUYER,0,1,3


In [6]:
# check why there was an error message about columns 18 and 19
azdias.iloc[:,19].unique(), azdias.iloc[:,20].unique()

(array([nan, 51.0, 24.0, 12.0, 43.0, 54.0, 22.0, 14.0, 13.0, 15.0, 33.0,
        41.0, 34.0, 55.0, 25.0, 23.0, 31.0, 52.0, 35.0, 45.0, 44.0, 32.0,
        '22', '24', '41', '12', '54', '51', '44', '35', '23', '25', '14',
        '34', '52', '55', '31', '32', '15', '13', '43', '33', '45', 'XX'], dtype=object),
 array([  2.,   5.,   3.,   4.,   1.,   6.,  nan]))

In [7]:
customers.iloc[:,19].unique(), customers.iloc[:,20].unique()

(array([13.0, nan, 34.0, 24.0, 41.0, 23.0, 15.0, 55.0, 14.0, 22.0, 43.0,
        51.0, 33.0, 25.0, 44.0, 54.0, 32.0, 12.0, 35.0, 31.0, 45.0, 52.0,
        '45', '25', '55', '51', '14', '54', '43', '22', '15', '24', '35',
        '23', '12', '44', '41', '52', '31', '13', '34', '32', '33', 'XX'], dtype=object),
 array([  5.,  nan,   2.,   6.,   4.,   1.,   3.]))

It seems values X, XX, and nan are the cause of the cause of the warnings received when loading the data. Later I will replace these values, but let's first learn about the values each attribute can take.

In [8]:
# load and view the attribute levels dataset
levels = pd.read_excel("DIAS Information Levels - Attributes 2017.xlsx", header = 1)
levels.head(10)

,Information level,Attribute,Description,Additional notes
NaN,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
NaN,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
NaN,NaN,ANREDE_KZ,gender,NaN
NaN,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
NaN,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...
NaN,NaN,FINANZ_SPARER,financial typology: money saver,NaN
NaN,NaN,FINANZ_VORSORGER,financial typology: be prepared,NaN
NaN,NaN,FINANZ_ANLEGER,financial typology: investor,NaN
NaN,NaN,FINANZ_UNAUFFAELLIGER,financial typology: unremarkable,NaN
NaN,NaN,FINANZ_HAUSBAUER,financial typology: main focus is the own house,NaN


In [9]:
#load the attibute values
attributes = pd.read_excel("DIAS Attributes - Values 2017.xlsx", header = 1, index = False)
attributes.head(15)

,Attribute,Description,Value,Meaning
NaN,AGER_TYP,best-ager typology,-1,unknown
NaN,NaN,NaN,0,no classification possible
NaN,NaN,NaN,1,passive elderly
NaN,NaN,NaN,2,cultural elderly
NaN,NaN,NaN,3,experience-driven elderly
NaN,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
NaN,NaN,NaN,1,< 30 years
NaN,NaN,NaN,2,30 - 45 years
NaN,NaN,NaN,3,46 - 60 years
NaN,NaN,NaN,4,> 60 years


The index contains NaNs, so I will replace it. Also, I will forward fill the "Atttribute" and "Description" columns as their existing values seem to be applicable until the next existing value.

In [10]:
# drop index
attributes.reset_index(drop=True, inplace=True) 
                 
# Forward fill the attribute and description columns
attributes['Attribute'] = attributes['Attribute'].ffill()
attributes['Description'] = attributes['Description'].ffill()

# see the first 15 rows
attributes.head(15)

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,AGER_TYP,best-ager typology,0,no classification possible
2,AGER_TYP,best-ager typology,1,passive elderly
3,AGER_TYP,best-ager typology,2,cultural elderly
4,AGER_TYP,best-ager typology,3,experience-driven elderly
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,ALTERSKATEGORIE_GROB,age classification through prename analysis,1,< 30 years
7,ALTERSKATEGORIE_GROB,age classification through prename analysis,2,30 - 45 years
8,ALTERSKATEGORIE_GROB,age classification through prename analysis,3,46 - 60 years
9,ALTERSKATEGORIE_GROB,age classification through prename analysis,4,> 60 years


Now that I know -1 consistently stands for unknown, which I will convert to NaN later, I will first replace X, XX, and nans detected earlier to -1.

In [11]:
def replace_values(df, cols):
    """
    Replaces 'X', 'XX' and 'nan' values with -1 in the general population and customer datasets. 
    Casts these columns as floats.
    
    Returns: Dataframe with updated columns
    """
    
    df.iloc[:, cols] = df.iloc[:, cols].replace({"X": -1, "XX": -1, "nan": -1})
    df.iloc[:, cols] = df.iloc[:, cols].astype(float)
    
    return df

In [12]:
# since loading datasets is time consuming, I first make copies of the datasets to be able to experiment with them
azdias_copy = azdias.copy()
customers_copy = customers.copy()

In [13]:
cols = [19, 20]
azdias_copy.iloc[:, cols].head()

,CAMEO_INTL_2015,CJT_GESAMTTYP
0,NaN,2.0
1,51,5.0
2,24,3.0
3,12,2.0
4,43,5.0


In [14]:
#apply the function to replace values in columns 18 and 19
cols = [19, 20]

azdias_copy = replace_values(azdias_copy, cols)
customers_copy = replace_values(customers_copy, cols)

In [15]:
# check if the values have been correctly replaced
azdias_copy.iloc[:,19].unique(), azdias_copy.iloc[:,20].unique(), customers_copy.iloc[:,19].unique(), customers_copy.iloc[:,20].unique()

(array([ nan,  51.,  24.,  12.,  43.,  54.,  22.,  14.,  13.,  15.,  33.,
         41.,  34.,  55.,  25.,  23.,  31.,  52.,  35.,  45.,  44.,  32.,
         -1.]),
 array([  2.,   5.,   3.,   4.,   1.,   6.,  nan]),
 array([ 13.,  nan,  34.,  24.,  41.,  23.,  15.,  55.,  14.,  22.,  43.,
         51.,  33.,  25.,  44.,  54.,  32.,  12.,  35.,  31.,  45.,  52.,
         -1.]),
 array([  5.,  nan,   2.,   6.,   4.,   1.,   3.]))

In [16]:
azdias_copy.iloc[:,19].sum()

28106233.0

**Convert unknowns to NaNs**

In [17]:
# attributes with uknown values
unknown_list = attributes[attributes["Meaning"] == "unknown"]

In [18]:
#number of attributes that contain unknown values
len(unknown_list)

232

In [32]:
unknown_list.head()

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
33,ANREDE_KZ,gender,"-1, 0",unknown
40,BALLRAUM,distance to next urban centre,-1,unknown
48,BIP_FLAG,business-flag indicating companies in the buil...,-1,unknown


In [90]:
def get_unknown_values(col, unknown_list):
    """
    Args: 
    col - columns to retrieve unknown values from
    df - Dataframe containing attribute values
    
    Returns unkown values for the selected columnn"""
    
    
    
    unknown_values = unknown_list[unknown_list["Attribute"] == col]["Value"]
    unknown_values = unknown_values.astype(str).str.cat(sep=",")
    unknown_values = [int(x) for x in unknown_values.split(",")]
      
    return set(unknown_values)

In [91]:
# test the function on a column
ZABEOTYP_unknown = get_unknown_values("ZABEOTYP", unknown_list)
ZABEOTYP_unknown

{-1, 9}

In [92]:
def replace_unknown_col(df, col, unknown_list):
    unknown_values = get_unknown_values(col, unknown_list)
    list2 = []
    for val in df[col].values:
        if val in unknown_values:
            list2.append(np.nan)
        else:
            list2.append(val)
    
    df[col] =  np.array(list2)
            
    return df[col]


In [111]:
def replace_unknowns(df, unknown_list, verbose=False):
    """
    Replaces unknown values of all columns to 'np.nan'
    """
    for col in unknown_list.Attribute:
        unknown_values = get_unknown_values(col, unknown_list)
        if verbose:
            print("Replacing {} to NaN in column {}".format(unknown_values, col))
        if col in df.columns:
            df[col] = replace_unknown_col(df, col, unknown_list)
    return df

In [112]:
replace_unknowns(azdias_copy, unknown_list, verbose = True)

Replacing {-1} to NaN in column AGER_TYP
Replacing {0, -1} to NaN in column ALTERSKATEGORIE_GROB
Replacing {0, -1} to NaN in column ANREDE_KZ
Replacing {-1} to NaN in column BALLRAUM
Replacing {-1} to NaN in column BIP_FLAG
Replacing {-1} to NaN in column CAMEO_DEUG_2015
Replacing {-1} to NaN in column CAMEO_DEUINTL_2015
Replacing {0} to NaN in column CJT_GESAMTTYP
Replacing {-1} to NaN in column D19_KK_KUNDENTYP
Replacing {-1} to NaN in column EWDICHTE
Replacing {-1} to NaN in column FINANZTYP
Replacing {-1} to NaN in column FINANZ_ANLEGER
Replacing {-1} to NaN in column FINANZ_HAUSBAUER
Replacing {-1} to NaN in column FINANZ_MINIMALIST
Replacing {-1} to NaN in column FINANZ_SPARER
Replacing {-1} to NaN in column FINANZ_UNAUFFAELLIGER
Replacing {-1} to NaN in column FINANZ_VORSORGER
Replacing {0, -1} to NaN in column GEBAEUDETYP
Replacing {0, -1} to NaN in column GEOSCORE_KLS7
Replacing {0, -1} to NaN in column HAUSHALTSSTRUKTUR
Replacing {-1} to NaN in column HEALTH_TYP
Replacing {0,

In [124]:
replace_unknowns(customers_copy, unknown_list, verbose = True)

Replacing {-1} to NaN in column AGER_TYP
Replacing {0, -1} to NaN in column ALTERSKATEGORIE_GROB
Replacing {0, -1} to NaN in column ANREDE_KZ
Replacing {-1} to NaN in column BALLRAUM
Replacing {-1} to NaN in column BIP_FLAG
Replacing {-1} to NaN in column CAMEO_DEUG_2015
Replacing {-1} to NaN in column CAMEO_DEUINTL_2015
Replacing {0} to NaN in column CJT_GESAMTTYP
Replacing {-1} to NaN in column D19_KK_KUNDENTYP
Replacing {-1} to NaN in column EWDICHTE
Replacing {-1} to NaN in column FINANZTYP
Replacing {-1} to NaN in column FINANZ_ANLEGER
Replacing {-1} to NaN in column FINANZ_HAUSBAUER
Replacing {-1} to NaN in column FINANZ_MINIMALIST
Replacing {-1} to NaN in column FINANZ_SPARER
Replacing {-1} to NaN in column FINANZ_UNAUFFAELLIGER
Replacing {-1} to NaN in column FINANZ_VORSORGER
Replacing {0, -1} to NaN in column GEBAEUDETYP
Replacing {0, -1} to NaN in column GEOSCORE_KLS7
Replacing {0, -1} to NaN in column HAUSHALTSSTRUKTUR
Replacing {-1} to NaN in column HEALTH_TYP
Replacing {0,

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,9626,2.0,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
1,9628,NaN,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
2,143872,NaN,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4
3,143873,1.0,1.0,8.0,NaN,NaN,NaN,NaN,8.0,0.0,...,2.0,NaN,9.0,7.0,1,COSMETIC,MULTI_BUYER,0,1,4
4,143874,NaN,1.0,20.0,NaN,NaN,NaN,NaN,14.0,7.0,...,4.0,2.0,9.0,3.0,1,FOOD,MULTI_BUYER,0,1,3
5,143888,1.0,1.0,11.0,NaN,NaN,NaN,NaN,10.0,1.0,...,1.0,6.0,9.0,1.0,2,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,3
6,143904,2.0,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,2.0,6.0,9.0,7.0,1,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
7,143910,1.0,1.0,10.0,NaN,NaN,NaN,NaN,9.0,1.0,...,1.0,6.0,9.0,3.0,3,FOOD,SINGLE_BUYER,0,1,4
8,102160,2.0,3.0,5.0,NaN,NaN,NaN,NaN,4.0,74.0,...,9.0,6.0,3.0,4.0,3,COSMETIC,MULTI_BUYER,0,2,4
9,102173,1.0,1.0,20.0,NaN,NaN,NaN,NaN,13.0,1.0,...,4.0,2.0,9.0,5.0,1,COSMETIC,MULTI_BUYER,0,1,3


In [125]:
#show columns that are of object type
customers_copy.loc[:, customers_copy.dtypes == object]

,CAMEO_DEU_2015,CAMEO_DEUG_2015,D19_LETZTER_KAUF_BRANCHE,EINGEFUEGT_AM,OST_WEST_KZ,PRODUCT_GROUP,CUSTOMER_GROUP
0,1A,1.0,D19_UNBEKANNT,1992-02-12 00:00:00,W,COSMETIC_AND_FOOD,MULTI_BUYER
1,NaN,nan,D19_BANKEN_GROSS,NaN,nan,FOOD,SINGLE_BUYER
2,5D,5.0,D19_UNBEKANNT,1992-02-10 00:00:00,W,COSMETIC_AND_FOOD,MULTI_BUYER
3,4C,4.0,D19_NAHRUNGSERGAENZUNG,1992-02-10 00:00:00,W,COSMETIC,MULTI_BUYER
4,7B,7.0,D19_SCHUHE,1992-02-12 00:00:00,W,FOOD,MULTI_BUYER
5,5D,5.0,D19_BUCH_CD,1992-02-10 00:00:00,W,COSMETIC_AND_FOOD,MULTI_BUYER
6,3B,3.0,D19_DROGERIEARTIKEL,1992-02-10 00:00:00,W,COSMETIC_AND_FOOD,MULTI_BUYER
7,1D,1.0,D19_SONSTIGE,1992-02-10 00:00:00,W,FOOD,SINGLE_BUYER
8,9E,9.0,D19_SONSTIGE,2004-10-14 00:00:00,W,COSMETIC,MULTI_BUYER
9,1D,1.0,D19_BUCH_CD,1997-07-14 00:00:00,W,COSMETIC,MULTI_BUYER


In [126]:
#show columns that are of object type
azdias_copy.loc[:, customers_copy.dtypes == object]

,CAMEO_DEU_2015,CAMEO_DEUG_2015,D19_LETZTER_KAUF_BRANCHE,EINGEFUEGT_AM,OST_WEST_KZ
0,NaN,nan,NaN,NaN,nan
1,8A,8.0,NaN,1992-02-10 00:00:00,W
2,4C,4.0,D19_UNBEKANNT,1992-02-12 00:00:00,W
3,2A,2.0,D19_UNBEKANNT,1997-04-21 00:00:00,W
4,6B,6.0,D19_SCHUHE,1992-02-12 00:00:00,W
5,8C,8.0,D19_ENERGIE,1992-02-12 00:00:00,W
6,4A,4.0,D19_UNBEKANNT,1992-02-12 00:00:00,W
7,2D,2.0,D19_UNBEKANNT,1992-02-10 00:00:00,W
8,1A,1.0,NaN,1992-02-10 00:00:00,W
9,1E,1.0,D19_KOSMETIK,1992-02-10 00:00:00,W


## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')